## Hands-on Tutorial: Quantifying and Reducing Gender Stereotypes in Word Embeddings

Ensuring fairness in algorithmically-driven decision-making is important to avoid inadvertent cases of bias and perpetuation of harmful stereotypes. However, modern natural language processing techniques, which learn model parameters based on data, might rely on implicit biases presented in the data to make undesirable stereotypical associations. Such a danger is facing us with word embedding, a popular framework to represent text data as vectors which has been used in many machine learning and natural language processing tasks. Recent results ([1](https://arxiv.org/abs/1607.06520), [2](https://arxiv.org/abs/1608.07187)) show that even word embeddings trained on Google News articles exhibit female/male gender stereotypes to a disturbing extent. This raises concerns because of their widespread use, as we describe, often tends to amplify these biases. 

In the following, we provide step-by-step instructions to demonstrate and quanitfy the biases in word embedding.



In [9]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [1]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [45]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [3]:
# gender direction
v_gender = E.diff('she', 'he')

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [4]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
she-he
herself-himself
her-his
woman-man
daughter-son
businesswoman-businessman
girl-boy
actress-actor
chairwoman-chairman
heroine-hero
mother-father
spokeswoman-spokesman
sister-brother
girls-boys
sisters-brothers
queen-king
niece-nephew
councilwoman-councilman
motherhood-fatherhood
women-men
petite-lanky
ovarian_cancer-prostate_cancer
Anne-John
schoolgirl-schoolboy
granddaughter-grandson
aunt-uncle
matriarch-patriarch
twin_sister-twin_brother
mom-dad
lesbian-gay
husband-younger_brother
gal-dude
lady-gentleman
sorority-fraternity
mothers-fathers
grandmother-grandfather
blouse-shirt
soprano-baritone
queens-kings
Jill-Greg
daughters-sons
grandma-grandpa
volleyball-football
diva-superstar
mommy-kid
Sarah-Matthew
hairdresser-barber
softball-baseball
goddess-god
Aisha-Jamal
waitress-waiter
princess-prince
filly-colt
mare-gelding
ladies-gentlemen
childhood-boyhood
interior_designer-architect
nun-priest
wig-beard
granddaughters-grandso

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [46]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23798442, 'maestro'),
  (-0.21665451, 'statesman'),
  (-0.20758669, 'skipper'),
  (-0.20267202, 'protege'),
  (-0.2020676, 'businessman'),
  (-0.19492392, 'sportsman'),
  (-0.18836352, 'philosopher'),
  (-0.1807366, 'marksman'),
  (-0.1728986, 'captain'),
  (-0.16785555, 'architect'),
  (-0.16702037, 'financier'),
  (-0.16313636, 'warrior'),
  (-0.15280862, 'major_leaguer'),
  (-0.15001445, 'trumpeter'),
  (-0.14718868, 'broadcaster'),
  (-0.14637242, 'magician'),
  (-0.14401694, 'fighter_pilot'),
  (-0.13782285, 'boss'),
  (-0.137182, 'industrialist'),
  (-0.13684885, 'pundit')],
 [(0.19714224, 'interior_designer'),
  (0.20833439, 'housekeeper'),
  (0.21560375, 'stylist'),
  (0.2236317, 'bookkeeper'),
  (0.23776126, 'maid'),
  (0.24125955, 'nun'),
  (0.24782579, 'nanny'),
  (0.24929334, 'hairdresser'),
  (0.24946158, 'paralegal'),
  (0.25276464, 'ballerina'),
  (0.25718823, 'socialite'),
  (0.26647124, 'librarian'),
  (0.27317622, 'receptionist'),
  (0.27540293, 'waitress'),
  (

## Part 2 Racial Bias

### Step 5: Define racial direction
We define racial direction based on the common names in different Demographic groups. 

In [53]:
# names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
#          "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
#          "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
#          "Leroy", "Brad", "Jermaine"]
# names_group1 = [names[2 * i] for i in range(len(names) // 2)]
# names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]

# names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
#          "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
#          "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
#          "Leroy", "Brad", "Jermaine"]


names = ["church", "mosque"]
# names = ["John", "Jin", "Anne", "Keisha", "Brett", "Lee", "Meredith", "Yujin"]

names_group1 = [names[2 * i] for i in range(len(names) // 2)]
names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]

print(names_group1, names_group2)

['church'] ['mosque']


In [54]:
# racial direction
vs = [sum(E.v(w) for w in names) for names in (names_group2, names_group1)]
vs = [v / np.linalg.norm(v) for v in vs]

v_racial = vs[1] - vs[0]
v_racial = v_racial / np.linalg.norm(v_racial)

### Step 6: Generating racial biased analogies

Similar to Step 3, we generate analogies that align with the racial dimension. 

In [55]:
# racial analogies
a_racial = E.best_analogies_dist_thresh(v_racial)

for (a,b,c) in a_racial:
    print(a+"-"+b)

churches-mosques
synagogue-mosque
church-synagogue
preacher-imam
priest-cleric
clergy-clerics
parishioners-worshipers
congregations-faiths
gospel-hip_hop
ministry-interior_ministry
pastors-preachers
hymns-verses
preachers-imams
evangelicals-fundamentalists
pastor-rabbi
orphanage-refugee_camp
Keisha-Aisha
parishes-counties
evangelical-fundamentalist
ecumenical-interfaith
linens-carpets
y-el
pianist-lyricist
choir-ensembles
humankind-civilizations
saints-martyrs
Jermaine-Kareem
quilting-embroidery
horses-camels
cathedral-mausoleum
diocesan-seminary
farm-slaughterhouse
orphanages-refugee_camps
parishioner-rector
gazebo-walkway
cashier-shopkeeper
musician-rapper
organist-violinist
doctrines-fundamentalism
troop-troop_surge
beans-onions
bishops-pontiff
supermarket-shopping_mall
chili-curry
denominations-religions
congregation-prayer
perpetrators-terrorists
berry-mango
joyful-jubilant
robe-turban
mudslides-flash_floods
tractor-bulldozer
patio-terrace
supper-dinner
dementia-colorectal_cancer


### Step 7: Analyzing racial bias in word vectors asscoiated with professions

Similar to Step 4, we project occpurations onto the racial dimension.

In [56]:
# profession analysis racial
sp = sorted([(E.v(w).dot(v_racial), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23647836, 'shopkeeper'),
  (-0.16741166, 'cleric'),
  (-0.16689804, 'cab_driver'),
  (-0.159675, 'taxi_driver'),
  (-0.14931153, 'fashion_designer'),
  (-0.13787618, 'cabbie'),
  (-0.13465147, 'protester'),
  (-0.12832956, 'financier'),
  (-0.12299342, 'cameraman'),
  (-0.11817284, 'policeman'),
  (-0.10585879, 'bodyguard'),
  (-0.1057793, 'lyricist'),
  (-0.10418746, 'journalist'),
  (-0.09905955, 'trader'),
  (-0.097547606, 'lawmaker'),
  (-0.09717408, 'fighter_pilot'),
  (-0.09258237, 'commander'),
  (-0.09019319, 'strategist'),
  (-0.089617535, 'firebrand'),
  (-0.089435786, 'officer')],
 [(0.12772599, 'foreman'),
  (0.12841928, 'guidance_counselor'),
  (0.13039456, 'homemaker'),
  (0.1333748, 'gardener'),
  (0.13884038, 'educator'),
  (0.1393606, 'handyman'),
  (0.14052024, 'registered_nurse'),
  (0.1450107, 'minister'),
  (0.15288307, 'bookkeeper'),
  (0.16230255, 'evangelist'),
  (0.16800667, 'soloist'),
  (0.16986589, 'pianist'),
  (0.1861384, 'nun'),
  (0.20823134, 'arch

## Exercise

Repeat Step 2-4 with debiased word embedding. 

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups. We give an example for the case of gender below for you to warm up.

In [10]:
from debiaswe.debias import debias

In [11]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [[u'woman', u'man'], [u'girl', u'boy'], [u'she', u'he'], [u'mother', u'father'], [u'daughter', u'son'], [u'gal', u'guy'], [u'female', u'male'], [u'her', u'his'], [u'herself', u'himself'], [u'Mary', u'John']]
gender specific 218 [u'actress', u'actresses', u'aunt', u'aunts', u'bachelor', u'ballerina', u'barbershop', u'baritone', u'beard', u'beards']


In [12]:
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
set([(u'Dad', u'Mom'), (u'fathers', u'mothers'), (u'Gelding', u'Mare'), (u'twin_brother', u'twin_sister'), (u'HIMSELF', u'HERSELF'), (u'GRANDSONS', u'GRANDDAUGHTERS'), (u'KING', u'QUEEN'), (u'FRATERNITY', u'SORORITY'), (u'prince', u'princess'), (u'men', u'women'), (u'FATHERHOOD', u'MOTHERHOOD'), (u'Dudes', u'Gals'), (u'DADS', u'MOMS'), (u'BOYS', u'GIRLS'), (u'nephew', u'niece'), (u'Father', u'Mother'), (u'He', u'She'), (u'Grandfather', u'Grandmother'), (u'Spokesman', u'Spokeswoman'), (u'Brother', u'Sister'), (u'FATHERS', u'MOTHERS'), (u'UNCLE', u'AUNT'), (u'gelding', u'mare'), (u'Himself', u'Herself'), (u'his', u'her'), (u'Son', u'Daughter'), (u'prostate_cancer', u'ovarian_cancer'), (u'BROTHER', u'SISTER'), (u'chairman', u'chairwoman'), (u'MEN', u'WOMEN'), (u'gentlemen', u'ladies'), (u'SON', u'DAUGHTER'), (u'king', u'queen'), (u'Colt', u'Filly'), (u'councilman', u'councilwoman'), (u'SPOKESMAN', u'SPOKESWO

In [13]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.41963255, u'congressman'),
  (-0.40675855, u'businessman'),
  (-0.32398781, u'councilman'),
  (-0.30967095, u'dad'),
  (-0.21665451, u'statesman'),
  (-0.11345412, u'salesman'),
  (-0.073004864, u'monk'),
  (-0.072163954, u'handyman'),
  (-0.049468268, u'minister'),
  (-0.043583866, u'archbishop'),
  (-0.040207233, u'bishop'),
  (-0.038332459, u'commissioner'),
  (-0.035724372, u'surgeon'),
  (-0.033134006, u'trader'),
  (-0.032377187, u'observer'),
  (-0.032095861, u'neurosurgeon'),
  (-0.031450123, u'priest'),
  (-0.031133929, u'skipper'),
  (-0.029659165, u'lawmaker'),
  (-0.029511193, u'commander')],
 [(0.029965654, u'teenager'),
  (0.030237058, u'instructor'),
  (0.030946141, u'student'),
  (0.031116983, u'paralegal'),
  (0.032039441, u'bookkeeper'),
  (0.032434631, u'cinematographer'),
  (0.034329094, u'graphic_designer'),
  (0.034705672, u'lifeguard'),
  (0.035666514, u'janitor'),
  (0.035971936, u'drummer'),
  (0.042120136, u'wrestler'),
  (0.043902256, u'hairdresser'),
 

In [14]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 10.2185974341
Median: 7.0
she-he
grandmother-grandfather
daughters-sons
women-men
princess-prince
mothers-fathers
females-males
woman-man
queen-king
councilwoman-councilman
gals-dudes
chairwoman-chairman
convent-monastery
female-male
granddaughters-grandsons
sorority-fraternity
moms-dads
niece-nephew
ovarian_cancer-prostate_cancer
grandma-grandpa
sisters-brothers
twin_sister-twin_brother
sister-brother
congresswoman-congressman
ex_boyfriend-ex_girlfriend
herself-himself
ladies-gentlemen
aunt-uncle
schoolgirl-schoolboy
businesswoman-businessman
motherhood-fatherhood
her-his
queens-kings
mother-father
granddaughter-grandson
spokeswoman-spokesman
mare-gelding
filly-colt
estrogen-testosterone
girls-boys
husbands-wives
daughter-son
girl-boy
mom-dad
actress-actor
lesbian-gay
compatriot-countryman
husband-younger_brother
gal-dude
hers-theirs
heroine-protagonist
feminism-feminist
actresses-actors
childhood-boyhood
waitress-waiter
kid-guy
me-him
mommy-daddy
aunts-uncle